


Dhruuv agarwal 

Denoising using 1d CNN



In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import math

In [0]:
def getSNR(yRawInpTimeDomain,yPredReadyToWrite):
    minLength = min(len(yRawInpTimeDomain),len(yPredReadyToWrite))
    clippedY_True = yRawInpTimeDomain[:minLength]
    clippedY_Pred = yPredReadyToWrite[:minLength]
    signal  = np.dot(clippedY_True,clippedY_True.T)
    subtract = clippedY_True - clippedY_Pred
    noise = np.dot(subtract,subtract.T)
    
    return 10*math.log10(signal/noise)

Here we get the files as given in the document and get the required input matrices after stft and absolute to get the magnitude.

We then take the transpose of these matrices to match the format of Input matrix in the graph structure.


In [0]:
!pip install librosa # in colab, you’ll need to install this
import librosa
s, sr=librosa.load('train_clean_male.wav', sr=None)
Sdata=librosa.stft(s, n_fft=1024, hop_length=512)
sn, sr=librosa.load('train_dirty_male.wav', sr=None)
Xdata1=librosa.stft(sn, n_fft=1024, hop_length=512)
Xdata2 = np.abs(Xdata1.T)
Sdata = np.abs(Sdata.T)

snt, srt=librosa.load('test_x_01.wav', sr=None)
Xtest=librosa.stft(snt, n_fft=1024, hop_length=512)
Xtestm = np.abs(Xtest.T)

snt2, srt2=librosa.load('test_x_02.wav', sr=None)
Xtest2=librosa.stft(snt2, n_fft=1024, hop_length=512)
Xtestm2 = np.abs(Xtest2.T)

print(Xdata2.shape)

FileNotFoundError: ignored

In [0]:
Xinp = tf.placeholder('float',[None,513])
yinp = tf.placeholder('float',[None,513])
y = tf.placeholder('float')
is_training = tf.placeholder(tf.bool)

from sklearn.model_selection import train_test_split

Xdata,Xval,Sdata,Sval = train_test_split(Xdata2,Sdata,test_size=0.2)

Below are the initial default parameters defined which will be changed later.

In [0]:
output = []
output2 = []
batch_size = 128

def cnn_model_fn(Xinp,yinp,mode):
  global output 
  global output2
  global batch_size
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(Xinp, [-1, 1,1,513])

  initconv = tf.contrib.layers.variance_scaling_initializer(uniform=False,factor=2.0)
  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[1, 9],
      padding="same",
      activation=None,
      data_format = 'channels_first'
  )#kernel_initializer=initconv)
  conv1_bn_relu = tf.nn.relu(conv1)
  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1_bn_relu, pool_size=[1, 2], strides=[1,2],data_format = 'channels_first')

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[1, 5],
      padding="same",
      activation=tf.nn.relu,
      data_format = 'channels_first'
  )#kernel_initializer=initconv)
  
  
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[1, 2], strides=[1,2],data_format = 'channels_first')
  init = tf.contrib.layers.variance_scaling_initializer(uniform=True,factor=1.0)
  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 1 * 128 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=2048, activation=tf.nn.relu)#,kernel_initializer=init,bias_initializer=init)#,data_format = 'channels_first')
  dropout = tf.layers.dropout(inputs=dense, rate=0.3,training=is_training)
  
  out = tf.layers.dense(inputs=dropout, units=513, activation=tf.nn.relu)#,kernel_initializer=init,bias_initializer=init)
  mse = tf.losses.mean_squared_error(labels=yinp, predictions = out)
  
  
  # Configure the Training Op (for TRAIN mode)
  
  optimizer = tf.train.AdamOptimizer(learning_rate=0.0004).minimize(mse)
    
  if mode == 'TRAIN':
    n_epochs = 1000


    with tf.Session() as sess:
      sess.run(tf.global_variables_initializer())
      
      for epoch in range(n_epochs):
        
        rand_index = np.random.choice(Xdata.shape[0], size=batch_size)
        np.random.shuffle(rand_index)
        res=[]
        #print(Xdata.shape[0])
        #print(rand_index)
        loss = 0
        for i in range(0,int(len(Xdata)/batch_size)):   
          vals =rand_index[i*batch_size:(i+1)*batch_size]
          xbatch = Xdata[vals]
          sbatch = Sdata[vals]
          res = sess.run([optimizer,mse],feed_dict={Xinp:xbatch,yinp:sbatch,is_training:True})
          loss+=res[1]
          last= (i+1)*batch_size
        
        xbatch = Xdata[rand_index[last:]]
        sbatch = Sdata[rand_index[last:]]
        #print(xbatch.shape)
        
        if len(xbatch)>1:
          res = sess.run([optimizer,mse],feed_dict={Xinp:xbatch,yinp:sbatch,is_training:True})
          loss += res[1]
        if epoch%10==0:
          print('Epoch',epoch,' out of ',n_epochs,' loss:',loss)
          tmp = sess.run(out,feed_dict = {Xinp:Xdata2, is_training:False})
          #print(Xdata2.shape,tmp.shape)
          sht = (Xdata1/Xdata2.T)*tmp.T
          iSt = librosa.istft(sht, hop_length=512)
          print("snr: ",getSNR(s,iSt))
      output = sess.run(out,feed_dict = {Xinp:Xtestm, is_training:False})
      output2 = sess.run(out,feed_dict = {Xinp:Xtestm2, is_training:False})
      print("val loss:",sess.run(mse,feed_dict = {Xinp:Xval, yinp:Sval,is_training:False}))
        #mseval = sess.run(mse,feed_dict = {Xinp:Xval,y:Sval})
      
 

In [0]:
#setparam(2,1024,513,513,100)
cnn_model_fn(Xinp,yinp,'TRAIN')
print(output2.shape)


We see the SNR ends up to be 14.5-15 approx in 1000 epochs. 

Also as we needed positive values at the end we have chosen relu activation at the last layer. 
The structure is 2 conv layer network with 1d filters (32 filters 1x9 and 64 filters 1x5) with maxpooling after both and 2048 units in following dense layer, finally a dense output layer. And the the leanrning rate was selected to be 0.0004. 

After this we get the Matrices learnt from the trained model and the Test input data, and convert them to get back the phase information as below.

This enables us to use istft and get the reconstructed version of the sound signal.

In [0]:
shtest1 = (Xtest/Xtestm.T)*output.T
shtest2 = (Xtest2/Xtestm2.T)*output2.T
print(shtest1.shape)

In [0]:
#snt, srt=librosa.load('test_x_01.wav', sr=None)
#snt2, srt2=librosa.load('test_x_02.wav', sr=None)

#Xtest=librosa.stft(snt, n_fft=1024, hop_length=512)
iStftMat1 = librosa.istft(shtest1, hop_length=512)
iStftMat2 = librosa.istft(shtest2, hop_length=512)


librosa.output.write_wav('test_s_01_recons.wav', iStftMat1, srt)
librosa.output.write_wav('test_s_02_recons.wav', iStftMat2, srt2)